# 如何强制代理调用工具

在此示例中，我们将构建一个 **始终** 调用某个工具的 ReAct 代理
首先，在制定任何计划之前。在此示例中，我们将创建一个代理
搜索工具。然而，一开始我们将强制代理调用搜索
工具（然后让它做任何它想做的事情）。当您知道时这很有用
您想要在应用程序中执行特定操作，但也希望
让法学硕士在完成后跟进用户的查询的灵活性
那个固定的顺序。

## 设置

首先我们需要安装需要的包
```bash
yarn add @langchain/langgraph @langchain/openai @langchain/core
```
接下来，我们需要为 OpenAI（我们将使用的 LLM）设置 API 密钥。可选地，我们
可以为[LangSmith追踪](https://smith.langchain.com/)设置API密钥，
将为我们提供一流的可观察性。

In [1]:
// process.env.OPENAI_API_KEY = "sk_...";

// 可选，在 LangSmith 中添加跟踪
// process.env.LANGCHAIN_API_KEY = "ls__...";
// process.env.LANGCHAIN_CALLBACKS_BACKGROUND = "true";
process.env.LANGCHAIN_TRACING_V2 = "true";
process.env.LANGCHAIN_PROJECT = "Force Calling a Tool First: LangGraphJS";

Force Calling a Tool First: LangGraphJS


## 设置工具

我们将首先定义我们想要使用的工具。对于这个简单的例子，我们将
通过 Tavily 使用内置搜索工具。但是，创建您的应用程序确实很容易
自己的工具 - 请参阅文档
[此处](https://js.langchain.com/docs/modules/agents/tools/dynamic)了解如何操作
那。

In [2]:
import { DynamicStructuredTool } from "@langchain/core/tools";
import { z } from "zod";

const searchTool = new DynamicStructuredTool({
  name: "search",
  description:
    "Use to surf the web, fetch current information, check the weather, and retrieve other information.",
  schema: z.object({
    query: z.string().describe("The query to use in your search."),
  }),
  func: async ({}: { query: string }) => {
    // 这是实际实现的占位符
    return "Cold, with a low of 13 ℃";
  },
});

await searchTool.invoke({ query: "What's the weather like?" });

const tools = [searchTool];

我们现在可以将这些工具包装在 `ToolNode` 中。
这是一个预先构建的节点，它接收 LangChain 聊天模型生成的工具调用并调用该工具，
返回输出。

In [3]:
import { ToolNode } from "@langchain/langgraph/prebuilt";

const toolNode = new ToolNode(tools);

## 设置模型

现在我们需要加载我们想要使用的聊天模型。\
重要的是，这应该满足两个标准：

1.它应该与消息一起使用。我们将以表格形式代表所有代理状态
消息，因此它需要能够与它们很好地配合。
2. 需要配合OpenAI函数调用。这意味着它应该是
OpenAI 模型或公开类似接口的模型。

注意：这些模型要求不是使用 LangGraph 的要求 - 它们
只是这个例子的要求。

In [4]:
import { ChatOpenAI } from "@langchain/openai";

const model = new ChatOpenAI({
  temperature: 0,
  model: "gpt-4o",
});

完成此操作后，我们应该确保模型知道它具有这些
可供调用的工具。我们可以通过将 LangChain 工具转换为
OpenAI函数调用的格式，然后将它们绑定到模型类。

In [5]:
const boundModel = model.bindTools(tools);

## 定义代理状态

`langgraph` 中图形的主要类型是 `StateGraph`。该图是
由传递到每个节点的状态对象参数化。每个节点
然后返回更新该状态的操作。

对于这个例子，我们将跟踪的状态只是一个消息列表。我们
希望每个节点只将消息添加到该列表中。因此，我们将定义
代理状态作为一个具有一个键（`messages`）的对象，其值指定如何
更新状态。

In [6]:
import { Annotation } from "@langchain/langgraph";
import { BaseMessage } from "@langchain/core/messages";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

## 定义节点

我们现在需要在图中定义一些不同的节点。在`langgraph`中，一个节点
可以是一个函数或一个
[可运行](https://js.langchain.com/docs/expression_language/)。有两个
为此我们需要的主要节点：

1. 代理：负责决定采取什么（如果有）行动。
2. 调用工具的函数：如果代理决定采取行动，该节点
然后将执行该操作。

我们还需要定义一些边。其中一些边缘可能是有条件的。
它们是有条件的原因是基于节点的输出，其中之一
可以采取多条路径。直到该节点才知道所采取的路径
运行（LLM 决定）。

1. 条件边缘：调用代理后，我们应该：如果
代理说要采取行动，那么调用工具的函数应该是
称为\
b.如果代理说已经完成，那么就应该完成
2. 正常边缘：调用工具后，应始终返回到
代理决定下一步做什么

让我们定义节点以及一个函数来决定什么条件
边取。

In [7]:
import { AIMessage, AIMessageChunk } from "@langchain/core/messages";
import { RunnableConfig } from "@langchain/core/runnables";
import { concat } from "@langchain/core/utils/stream";

// 定义将用于确定要下降的条件边沿的逻辑
const shouldContinue = (state: typeof AgentState.State) => {
  const { messages } = state;
  const lastMessage = messages[messages.length - 1] as AIMessage;
  // 如果没有函数调用，那么我们就完成了
  if (!lastMessage.tool_calls || lastMessage.tool_calls.length === 0) {
    return "end";
  }
  // 否则，如果有，我们继续
  return "continue";
};

// 定义调用模型的函数
const callModel = async (
  state: typeof AgentState.State,
  config?: RunnableConfig,
) => {
  const { messages } = state;
  let response: AIMessageChunk | undefined;
  for await (const message of await boundModel.stream(messages, config)) {
    if (!response) {
      response = message;
    } else {
      response = concat(response, message);
    }
  }
  // 我们返回一个对象，因为这将被添加到现有列表中
  return {
    messages: response ? [response as AIMessage] : [],
  };
};

**修改**

在这里，我们创建一个通过工具调用返回 AIMessage 的节点 - 我们将使用
这在开始时强制它调用一个工具

In [8]:
// 这是新的第一次 - 我们想要显式硬编码某些操作的模型的第一次调用
const firstModel = async (state: typeof AgentState.State) => {
  const humanInput = state.messages[state.messages.length - 1].content || "";
  return {
    messages: [
      new AIMessage({
        content: "",
        tool_calls: [
          {
            name: "search",
            args: {
              query: humanInput,
            },
            id: "tool_abcd123",
          },
        ],
      }),
    ],
  };
};

## 定义图表

我们现在可以将它们放在一起并定义图表！

**修改**

我们将定义一个 `firstModel` 节点，将其设置为入口点。


In [9]:
import { END, START, StateGraph } from "@langchain/langgraph";

// 定义一个新图
const workflow = new StateGraph(AgentState)
  // 定义新的入口点
  .addNode("first_agent", firstModel)
  // 定义我们将在其之间循环的两个节点
  .addNode("agent", callModel)
  .addNode("action", toolNode)
  // 将入口点设置为“first_agent”
  // 通过创建从虚拟 __start__ 节点到 `first_agent` 的边
  .addEdge(START, "first_agent")
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“代理”。
    // 这意味着这些是调用“agent”节点后获取的边。
    "agent",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinue,
    // 最后我们传入一个映射。
    // 键是字符串，值是其他节点。
    // END 是一个特殊的节点，标记图形应该结束。
    // 将会发生的事情是我们将调用“should_continue”，然后该输出
    // 将与此映射中的键进行匹配。
    // 根据匹配的节点，将调用该节点。
    {
      // 如果是“tools”，那么我们称之为工具节点。
      continue: "action",
      // 否则我们就结束了。
      end: END,
    },
  )
  // 我们现在添加从“tools”到“agent”的正常边缘。
  // 这意味着在调用“tools”之后，接下来调用“agent”节点。
  .addEdge("action", "agent")
  // 在我们致电第一个代理后，我们知道我们想要采取行动
  .addEdge("first_agent", "action");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const app = workflow.compile();

## 使用它！

我们现在可以使用它了！现在这暴露了
[相同的界面](https://js.langchain.com/docs/expression_language/) 与所有
其他 LangChain 运行程序。

In [10]:
import { HumanMessage } from "@langchain/core/messages";

const inputs = {
  messages: [new HumanMessage("what is the weather in sf")],
};

for await (const output of await app.stream(inputs)) {
  console.log(output);
  console.log("-----\n");
}

{
  first_agent: {
    messages: [
      AIMessage {
        "content": "",
        "additional_kwargs": {},
        "response_metadata": {},
        "tool_calls": [
          {
            "name": "search",
            "args": {
              "query": "what is the weather in sf"
            },
            "id": "tool_abcd123"
          }
        ],
        "invalid_tool_calls": []
      }
    ]
  }
}
-----

{
  action: {
    messages: [
      ToolMessage {
        "content": "Cold, with a low of 13 ℃",
        "name": "search",
        "additional_kwargs": {},
        "response_metadata": {},
        "tool_call_id": "tool_abcd123"
      }
    ]
  }
}
-----

{
  agent: {
    messages: [
      AIMessageChunk {
        "id": "chatcmpl-9y562g16z0MUNBJcS6nKMsDuFMRsS",
        "content": "The current weather in San Francisco is cold, with a low of 13°C.",
        "additional_kwargs": {},
        "response_metadata": {
          "prompt": 0,
          "completion": 0,
          "finish_reaso